## 결과


> #1 Age, Cabin, Ticket 제거 valid 0.819 lb 0.747

> #2 Cabin, Ticket 제거, Age 양자화 후 보간 valid 0.833 lb 0.730

> #3 Cabin, Ticket 제거, Age 평균 보간 : valid 0.866 lb 0.746

> #4 Age, Ticket 제거, Cabin 카테고리 분류 : valid 0.819 lb 0.762


## 코드

In [8]:
import pandas as pd
import numpy as np

In [486]:
from catboost import CatBoostClassifier, CatBoostRegressor
import catboost as cb

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

In [32]:
train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,NaN,2.0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,0.0
2,3,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,2.0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,C123,2.0
4,5,0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,NaN,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",0,27.0,0,0,211536,13.0000,NaN,2.0
887,888,1,1,"Graham, Miss. Margaret Edith",1,19.0,0,0,112053,30.0000,B42,2.0
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",1,NaN,1,2,W./C. 6607,23.4500,NaN,2.0
889,890,1,1,"Behr, Mr. Karl Howell",0,26.0,0,0,111369,30.0000,C148,0.0


In [62]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submit = pd.read_csv('submission.csv')

train['Sex'] = train['Sex'].replace('male', 0)
train['Sex'] = train['Sex'].replace('female', 1)
train['Embarked'] = train['Embarked'].replace('C', 0)
train['Embarked'] = train['Embarked'].replace('Q', 1)
train['Embarked'] = train['Embarked'].replace('S', 2)
train.drop(columns=['PassengerId', 'Name'], inplace=True)

test['Sex'] = test['Sex'].replace('male',0)
test['Sex'] = test['Sex'].replace('female',1)
test['Embarked'] = test['Embarked'].replace('C', 0)
test['Embarked'] = test['Embarked'].replace('Q', 1)
test['Embarked'] = test['Embarked'].replace('S', 2)
test.drop(columns=['PassengerId', 'Name'], inplace=True)

test.loc[test[test.Fare.isnull()].index, 'Fare'] = 8.0500

In [405]:
submit

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


## #1 Age, Cabin, Ticket 제거

In [452]:
_train, valid = train_test_split(train, test_size = 0.1, random_state=42)

train_x, train_y = _train.drop(columns=['Survived']), _train['Survived']
valid_x, valid_y = valid.drop(columns=['Survived']), valid['Survived']
_test = test.copy()

for data in [train_x, valid_x, _test]:
    data.drop(columns=['Age', 'Cabin', 'Ticket'], inplace=True)

train_dataset = cb.Pool(train_x, train_y) 
valid_dataset = cb.Pool(valid_x, valid_y)
test_dataset = cb.Pool(_test)

model = cb.CatBoostClassifier(loss_function='Logloss', silent=True, random_state=42)
model.fit(train_dataset)

pred = model.predict(valid_dataset)
score = roc_auc_score(valid_y.values, pred)
print(score)

submit['Survived'] = model.predict(test_dataset)
submit.to_csv('#1.csv', index=False)

0.8194444444444445


## #2 Cabin, Ticket 제거, Age 양자화 후 보간

In [411]:
_train, valid = train_test_split(train, test_size = 0.1, random_state=42)

_train = train.copy()
best_score = 0
related_cols = ['Pclass', 'SibSp', 'Parch']
for board in range(1, len(related_cols)+1):
    cols = related_cols[:board]
    print(cols)
    
    for qnt in range(2, 11):
        _train.Age = train.Age.copy()
        _train.Age = pd.cut(train.Age, qnt, labels=range(qnt))
        table = _train[cols + ['Age']].value_counts().sort_index()
        
        for index, *value in _train.loc[_train.Age.isnull(), cols].reset_index().values:
            for i in range(len(cols)):
                if (table.index.get_level_values(i) == value[i]).any():
                    continue
                while not (table.index.get_level_values(i) == value[i]).any():
                    value[i] -= 1
                break
            
            value = tuple(value)
            age = table.loc[value].sort_values(ascending=False).index[0]
            _train.loc[index, 'Age'] = age
        
        _train.Age = _train.Age.astype(int)
        
        train_x, train_y = _train.drop(columns=['Survived']), _train['Survived']
        valid_x, valid_y = valid.drop(columns=['Survived']), valid['Survived']
        _test = test.copy()

        for data in [train_x, valid_x, _test]:
            data.drop(columns=['Cabin', 'Ticket'], inplace=True)

        train_dataset = cb.Pool(train_x, train_y) 
        valid_dataset = cb.Pool(valid_x, valid_y)
        test_dataset = cb.Pool(_test)

        model = cb.CatBoostClassifier(loss_function='Logloss', silent=True, random_state=42)
        model.fit(train_dataset)

        pred = model.predict(valid_dataset)
        score = roc_auc_score(valid_y.values, pred)
        print(qnt, score)
        
        if best_score < score:
            best_comb = (cols, qnt)
            best_score = score
            submit['Survived'] = model.predict(test_dataset)
            submit.to_csv(f'#2.csv', index=False)
            
print("best is ", best_comb)

['Pclass']
2 0.7962962962962963
3 0.7916666666666665
4 0.8333333333333333
5 0.7777777777777777
6 0.7129629629629629
7 0.7777777777777777
8 0.6898148148148148
9 0.75
10 0.7592592592592592
['Pclass', 'SibSp']
2 0.7962962962962963
3 0.7916666666666665
4 0.8333333333333333
5 0.7777777777777777
6 0.7129629629629629
7 0.7268518518518519
8 0.6898148148148148
9 0.75
10 0.7453703703703703
['Pclass', 'SibSp', 'Parch']
2 0.7962962962962963
3 0.7916666666666665
4 0.8333333333333333
5 0.7777777777777777
6 0.7268518518518519
7 0.75
8 0.7083333333333334
9 0.8009259259259258
10 0.8055555555555555
best is  (['Pclass'], 4)


## #3 Cabin, Ticket 제거, Age 평균 보간

In [453]:
_train, valid = train_test_split(train, test_size = 0.1, random_state=42)

table = _train.pivot_table(values = 'Age', index = 'Pclass', aggfunc = 'mean')
for class_num in [1, 2, 3]:
    _train.loc[_train.loc[_train.Age.isnull()].loc[_train.Pclass == class_num].index]['Age'] = table.loc[class_num]

train_x, train_y = _train.drop(columns=['Survived']), _train['Survived']
valid_x, valid_y = valid.drop(columns=['Survived']), valid['Survived']
_test = test.copy()

for data in [train_x, valid_x, _test]:
    data.drop(columns=['Cabin', 'Ticket'], inplace=True)

train_dataset = cb.Pool(train_x, train_y) 
valid_dataset = cb.Pool(valid_x, valid_y)
test_dataset = cb.Pool(_test)

model = cb.CatBoostClassifier(loss_function='Logloss', silent=True, random_state=42)
model.fit(train_dataset)

pred = model.predict(valid_dataset)
score = roc_auc_score(valid_y.values, pred)
print(score)

submit['Survived'] = model.predict(test_dataset)
submit.to_csv('#3.csv', index=False)

0.8657407407407408


## #4 Age, Ticket 제거, Cabin 카테고리 분류

In [469]:
_train, valid = train_test_split(train, test_size = 0.1, random_state=42)
_train['Cabin'] = train['Cabin'].astype('category').cat.codes
_train

/tmp/ipykernel_885/3386461707.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _train['Cabin'] = train['Cabin'].astype('category').cat.codes


,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
165,1,3,0,9.0,0,2,363291,20.5250,-1,2.0
541,0,3,1,9.0,4,2,347082,31.2750,-1,2.0
625,0,1,0,61.0,0,0,36963,32.3208,110,2.0
388,0,3,0,NaN,0,0,367655,7.7292,-1,1.0
76,0,3,0,NaN,0,0,349208,7.8958,-1,2.0
...,...,...,...,...,...,...,...,...,...,...
106,1,3,1,21.0,0,0,343120,7.6500,-1,2.0
270,0,1,0,NaN,0,0,113798,31.0000,-1,2.0
860,0,3,0,41.0,2,0,350026,14.1083,-1,2.0
435,1,1,1,14.0,1,2,113760,120.0000,47,2.0


In [513]:
_train = train.copy()
_train['Cabin'] = train['Cabin'].astype('category').cat.codes

cabin_train, cabin_test = _train[_train.Cabin != -1], _train[_train.Cabin == -1]
cabin_dataset = cb.Pool(cabin_train.drop(columns=['Survived', 'Cabin', 'Age', 'Ticket']), cabin_train['Cabin'])

model = cb.CatBoostClassifier(loss_function='MultiClass', silent=True, random_state=42)
model.fit(cabin_dataset)

pred = model.predict(cabin_test.drop(columns=['Survived', 'Cabin', 'Age', 'Ticket']))
_train.loc[cabin_test.index]['Cabin'] = np.reshape(pred, (-1,))

_train, valid = train_test_split(_train, test_size = 0.1, random_state=42)

train_x, train_y = _train.drop(columns=['Survived']), _train['Survived']
valid_x, valid_y = valid.drop(columns=['Survived']), valid['Survived']
_test = test.copy()
_test['Cabin'] = test['Cabin'].astype('category').cat.codes

for data in [train_x, valid_x, _test]:
    data.drop(columns=['Age', 'Ticket'], inplace=True)

train_dataset = cb.Pool(train_x, train_y) 
valid_dataset = cb.Pool(valid_x, valid_y)
test_dataset = cb.Pool(_test)

model = cb.CatBoostClassifier(loss_function='Logloss', silent=True, random_state=42)
model.fit(train_dataset)

pred = model.predict(valid_dataset)
score = roc_auc_score(valid_y.values, pred)
print(score)

submit['Survived'] = model.predict(test_dataset)
submit.to_csv('#4.csv', index=False)

0.8194444444444445
